# False Positives

In the previous analysis we have checked if TE would correctly identify lag by computing TE at multiple competing  lags and finding the lag argument with maximal lag.

In this study we will be more strict. We will verify if true instantainous lag can be misinterpreted as a signle lag

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mesostat.metric.metric import MetricCalculator

from models import two_node_system

def crosscorr(x, y):
    N = x.shape[0]
    return np.correlate(x, y, 'full')[N - 1:] / N

In [ ]:
data2D = two_node_system(1000, [0], [1.0], noiseFrac=0.1, convDT=0.05, convTau=0.5)

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(8,4))
ax[0].plot(data2D[0], label='x')
ax[0].plot(data2D[1]+1, label='y')
ax[0].set_title('Signals')
ax[0].legend()
ax[1].plot(crosscorr(data2D[0], data2D[1])[:100])
ax[1].set_title('Cross-correlation')
plt.show()

In [ ]:
def crosscorr(x, y):
    N = x.shape[0]
    return np.correlate(x, y, 'full')[N - 1:] / N

idtxlParam = {
    'dim_order'       : 'ps',
    'cmi_estimator'   : 'JidtGaussianCMI',
    'max_lag_sources' : 1,
    'min_lag_sources' : 1,
    'parallelTrg' : False
}

In [ ]:
noiseFracLst = 10**np.linspace(-3, -0.5, 15)

for signalCorr in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    print("Correlation fraction", signalCorr)
    teLstMu = []
    pvalLstMu = []
    teLstStd = []
    pvalLstStd = []
    isSign = []

    for noiseFrac in noiseFracLst:
    #     print(noiseFrac)
        teTmpLst = []
        pvalTmpLst = []
        for iTest in range(100):
            data2D = two_node_system(1000, [0], [signalCorr], noiseFrac=noiseFrac, convDT=0.05, convTau=0.5)

            mc.set_data(data2D, 'ps')
            rez = mc.metric3D('BivariateTE', '', metricSettings=idtxlParam)

#             noiseFracLstEff += [noiseFrac] * 2
            teTmpLst += [rez[0,0,1]] + [rez[0,1,0]]
            pvalTmpLst += [rez[2,0,1]] + [rez[2,1,0]]

        teLstMu += [np.nanmean(teTmpLst)]    
        pvalLstMu += [np.nanmean(pvalTmpLst)]
        teLstStd += [np.nanstd(teTmpLst)]    
        pvalLstStd += [np.nanstd(pvalTmpLst)]

        pvalTmpLst = np.array(pvalTmpLst)
        pvalTmpLst[np.isnan(pvalTmpLst)] = 1
        isSign += [np.mean(pvalTmpLst < 0.01)]

    fig, ax = plt.subplots(ncols=3, figsize=(12,4))
    ax[0].errorbar(noiseFracLst, teLstMu, teLstStd)
    ax[1].errorbar(noiseFracLst, pvalLstMu, pvalLstStd)
    ax[2].semilogx(noiseFracLst, isSign)
    ax[0].set_title('TE')
    ax[1].set_title('pval')
    ax[2].set_title('Fraction significant')

    ax[0].set_xlabel('Noise-to-signal ratio')
    ax[1].set_xlabel('Noise-to-signal ratio')
    ax[2].set_xlabel('Noise-to-signal ratio')


    ax[0].set_yscale('log')
    ax[0].set_xscale('log')
    ax[1].set_xscale('log')
    plt.show()